# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,badger,64.8000,-147.5333,16.15,72,0,2.06,US,1722797046
1,1,kirakira,-10.4544,161.9205,26.00,87,100,5.23,SB,1722797547
2,2,waitangi,-43.9535,-176.5597,6.01,89,100,6.61,NZ,1722796967
3,3,bethel,41.3712,-73.4140,29.00,74,40,3.60,US,1722796945
4,4,iqaluit,63.7506,-68.5145,5.85,93,100,2.06,CA,1722797550


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Create a dictionary to store city, latitude, longitude and humidity values from city_data_df
city_humidity_data_coord = {
    "City": city_data_df["City"],
    "Latitude": city_data_df["Lat"],
    "Longitude": city_data_df["Lng"],
    "Humidity": city_data_df["Humidity"]
}

# Store the coordinates and humidity values in a DataFrame
city_humidity_data_coord_df = pd.DataFrame(city_humidity_data_coord)

# Configure the map plot
map_plot = city_humidity_data_coord_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.8,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# Narrow down city_data_df to ideal conditions where 'Max Temp' is between 29.444 and 32 degrees Celsius 

filtered_city_data_df = city_data_df.loc[(city_data_df["Max Temp"] > 29.444) & (city_data_df["Max Temp"] < 30.5)]

# Drop any rows with null values
filtered_city_data_df = filtered_city_data_df.dropna()

# Display sample data
filtered_city_data_df

# Export the filtered city data into a CSV file
filtered_city_data_df.to_csv("../output_data/filtered_cities.csv", index = True)


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Import CSV file results of filtered_cities.csv to hotel_df 
hotel_df = pd.read_csv("../output_data/filtered_cities.csv")

# Clean up hotel_df to only show the columns "City," "Country," "Lat," "Lng," and "Max Temp"
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Max Temp"]]
                        
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Max Temp,Hotel Name
0,chinna salem,IN,11.6500,78.9000,29.83,
1,la seu d'urgell,ES,42.3588,1.4614,29.56,
2,ormond-by-the-sea,US,29.3491,-81.0665,30.03,
3,huangmei,CN,30.1924,116.0250,29.51,
4,al jawf,SA,29.5000,38.7500,30.21,
5,vega de alatorre,MX,20.0333,-96.6333,29.66,
6,roses,ES,42.2620,3.1769,29.62,
7,sunam,IN,30.1333,75.8000,29.71,
8,watertown,US,42.3709,-71.1828,30.23,
9,plymouth,US,41.8334,-70.8328,29.82,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
limit = 1
radius = 100000
params = {
    "apiKey": geoapify_key,
    "limit": limit,
    "radius": radius
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index, "Lat"]
    lng = hotel_df.loc[index, "Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    params["categories"] = "accommodation.hotel"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Narrow down the hotel_df to only 10 cities records where a hotel was found
hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != "No hotel found"]

# Display sample data
hotel_df

Starting hotel search
chinna salem - nearest hotel: Cylone Hotel
la seu d'urgell - nearest hotel: Parador de la Seu d'Urgell
ormond-by-the-sea - nearest hotel: Howard Johnson
huangmei - nearest hotel: No hotel found
al jawf - nearest hotel: No hotel found
vega de alatorre - nearest hotel: Hotel Istirincha
roses - nearest hotel: Ramblamar
sunam - nearest hotel: Sunrize Palace
watertown - nearest hotel: Four Points by Sheraton
plymouth - nearest hotel: Quality Inn
gao - nearest hotel: No hotel found
inirida - nearest hotel: Residencias El Turista del Guainia
santa isabel - nearest hotel: No hotel found


,City,Country,Lat,Lng,Max Temp,Hotel Name
0,chinna salem,IN,11.6500,78.9000,29.83,Cylone Hotel
1,la seu d'urgell,ES,42.3588,1.4614,29.56,Parador de la Seu d'Urgell
2,ormond-by-the-sea,US,29.3491,-81.0665,30.03,Howard Johnson
5,vega de alatorre,MX,20.0333,-96.6333,29.66,Hotel Istirincha
6,roses,ES,42.2620,3.1769,29.62,Ramblamar
7,sunam,IN,30.1333,75.8000,29.71,Sunrize Palace
8,watertown,US,42.3709,-71.1828,30.23,Four Points by Sheraton
9,plymouth,US,41.8334,-70.8328,29.82,Quality Inn
11,inirida,CO,3.8653,-67.9239,29.83,Residencias El Turista del Guainia


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Create a dictionary to store city, latitude, longitude and humidity values from city_data_df

city_humidity_data_coord_2 = {
    "City": hotel_df["City"],
    "Country": hotel_df["Country"],
    "Latitude": hotel_df["Lat"],
    "Longitude": hotel_df["Lng"],
    "Max Temp (Celsius)": hotel_df["Max Temp"],
}

# Store the coordinates and humidity values in a DataFrame
city_humidity_data_coord_df_2 = pd.DataFrame(city_humidity_data_coord_2)

# Add hotel name next to the city name and country in the legend

city_humidity_data_coord_df_2["City"] = city_humidity_data_coord_df_2["City"].str.capitalize() + " - " + hotel_df["Country"] + " - " + hotel_df["Hotel Name"]

# Configure the map plot
map_plot_2 = city_humidity_data_coord_df_2.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 700,
    size = "Max Temp (Celsius)",
    scale = 2,
    color = "City"
)

# Display the map
map_plot_2


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (City,Max Temp (Celsius))